In [7]:
#!pip install requests
#!pip3 install Beautifulsoup4

In [192]:
import requests
from bs4 import BeautifulSoup
from math import *


In [2]:
URL = 'https://www.airlinequality.com/review-pages/a-z-airline-reviews/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
tabs_content = soup.find('div', class_="tabs-content")

In [5]:
airlines_tag = tabs_content.find_all('li')

In [6]:
#collect the different url for each airline company
airlines_ref=[]
for element in airlines_tag :
    tag = element.find('a')
    airlines_ref.append(tag['href'])
    

In [205]:
len(airlines_ref)

519

In [200]:
test = ['/airline-reviews/ab-aviation','/airline-reviews/air-france']

In [204]:
def get_url_airline(airlines):
    
    """
    Returns the sum of two decimal numbers in binary digits.

    Parameters
    ----------
    airlines (list): A list of airlines reference needed, ex : '/airline-reviews/ab-aviation'

    Returns
    -------
    A list of all url reviews linked to the companies needed   
    """
    
    URL = 'https://www.airlinequality.com'
    
    total_url=[]
    
    for airline in airlines :
        
        url_company ={'airlines':[],'seats':[]}
        url = URL + airline
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        #Test if onglet seat and airline
        seat_content = soup.find('li', class_=["tab-title seat","tab-title seat active"])
        airline_content = soup.find('li', class_=["tab-title airline","tab-title airline active"])
        
        if not(airline_content == None):
            ref_airline = airline_content.find('a')['href']
            new_url = URL + ref_airline + '/?sortby=post_date%3ADesc&pagesize=100'
            new_page = requests.get(new_url)
            new_soup = BeautifulSoup(new_page.content, 'html.parser')
            
            test_page=new_soup.find(
                    'article',class_="comp comp_reviews-pagination querylist-pagination position-")
            
            airline_url = []
            if not(test_page == None):
                query_page = new_soup.find('div', class_='pagination-total').get_text()
                
                if not(query_page ==None):
                    number_review = int(query_page.split(" ")[-2])
                    number_page = ceil(number_review/100)

                    for i in range(number_page) :
                        url = URL + ref_airline + '/page/'+ str(i+1) + '/?sortby=post_date%3ADesc&pagesize=100'
                        if url not in airline_url:
                            airline_url.append(url)
                        
            else : airline_url.append(new_url) #only one page
        
            url_company['airlines'].append(airline_url)

        if not(seat_content == None):
            ref_seat = seat_content.find('a')['href']
            new_url = URL + ref_seat + '/?sortby=post_date%3ADesc&pagesize=100'
            new_page = requests.get(new_url)
            new_soup = BeautifulSoup(new_page.content, 'html.parser')
            
            test_page=new_soup.find(
                    'article',class_="comp comp_reviews-pagination querylist-pagination position-")
            
            seat_url = []
            if not(test_page == None):
                query_page = new_soup.find('div', class_='pagination-total').get_text()
                
                if not(query_page ==None):
                    number_review = int(query_page.split(" ")[-2])
                    number_page = ceil(number_review/100)

                    for i in range(number_page) :
                        url = URL + ref_seat + '/page/'+ str(i+1) + '/?sortby=post_date%3ADesc&pagesize=100'
                        if url not in seat_url:
                            seat_url.append(url)
                        
            else : seat_url.append(new_url) # only one page
                
            url_company['seats'].append(seat_url)
            
        total_url.append(url_company)
    return total_url

get_url_airline(airlines_ref)

[{'airlines': [['https://www.airlinequality.com/airline-reviews/ab-aviation/?sortby=post_date%3ADesc&pagesize=100']],
  'seats': []},
 {'airlines': [['https://www.airlinequality.com/airline-reviews/adria-airways/?sortby=post_date%3ADesc&pagesize=100']],
  'seats': []},
 {'airlines': [['https://www.airlinequality.com/airline-reviews/aegean-airlines/page/1/?sortby=post_date%3ADesc&pagesize=100',
    'https://www.airlinequality.com/airline-reviews/aegean-airlines/page/2/?sortby=post_date%3ADesc&pagesize=100',
    'https://www.airlinequality.com/airline-reviews/aegean-airlines/page/3/?sortby=post_date%3ADesc&pagesize=100',
    'https://www.airlinequality.com/airline-reviews/aegean-airlines/page/4/?sortby=post_date%3ADesc&pagesize=100',
    'https://www.airlinequality.com/airline-reviews/aegean-airlines/page/5/?sortby=post_date%3ADesc&pagesize=100',
    'https://www.airlinequality.com/airline-reviews/aegean-airlines/page/6/?sortby=post_date%3ADesc&pagesize=100',
    'https://www.airlinequal

In [22]:

URL1 = 'https://www.airlinequality.com'
dict_reviews={}
for airline in airlines_ref:
    new_url = URL1 + airline
    new_page = requests.get(new_url)
    new_soup = BeautifulSoup(new_page.content, 'html.parser')
    
    seat_content = new_soup.find('li', class_=["tabs-title seat","tabs-title seat active"])
    print(seat_content)
    print(airline.split('/')[-1])
    if not(seat_content == None):
     #   dict_reviews[airline.split('/')[-1]]=[]
        
      #  col_content = soup.find('div', class_='col-content'
        
        
    break
    
    

None
ab-aviation


In [141]:
airline='/airline-reviews/air-france'

url2 = URL1 + airline
new_page = requests.get(url2)
new_soup = BeautifulSoup(new_page.content, 'html.parser')


In [158]:
seat_content = new_soup.find('li', class_="tab-title seat")
if not(seat_content == None):
    dict_reviews[airline.split('/')[-1]]=[]
    
    url3 = URL1 + seat_content.find('a')['href'] +
    page2 = requests.get(url3)
    soup2 = BeautifulSoup(page2.content, 'html.parser')
    
    #col_content = soup2.find('div', id='container')
    col_content = soup2.find('section', class_="layout-section layout-2 closer-top")
    col_content = col_content.find('div', class_='col-content')

    #col_content = col_content.find('div', class_='layout-section layout-2 closer-top')

    #col_content = col_content.find('article', class_="comp comp_reviews-airline querylist position-content ")
    
    review = col_content.find_all('div', class_="text_content")
    
    for comment in review :
        dict_reviews[airline.split('/')[-1]].append(comment.get_text())
        
print(dict_reviews)


{'air-france': ["✅ Trip Verified |  The business class seat had plenty of room, and I liked that it was a pod-style (so I had decent privacy). My biggest complaint is the comfort when in bed mode it is very lumpy with the lumbar protruding into your back.  AF has invested a lot into these seats, and they could be great if they would only take the time to work on them a bit. A lay-flat bed that protrudes into your back is not OK. Tried using an extra blanket to make the bed more comfy but it didn't work. Please Air France, improve the comfort of the cushions for bed mode.", '✅ Trip Verified |  I had done online check in for economy seats fairly forward but at boarding gate in paris they changed the seat to somewhere at the back where the planes width narrows. The seat was so small & rigid & once the guy in front of me reclined his seat i felt so claustrophobic. For the sake on entertainment in an 11hr plus flight which was further delayed by 2hrs was a tiny screen with little content & 

In [152]:
len(review)

10

In [154]:
dict_reviews

{'air-france': [None, None, None, None, None, None, None, None, None, None]}

In [ ]:
if not(seat_content == None):
    dict_reviews[airline.split('/')[-1]]=[]

    col_content = soup.find('div', class_='col-content')

print(col_content)